In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel

In [2]:
# input and output folders
trainingData = "ex_dataText/trainingData.csv"
unlabeledData = "ex_dataText/unlabeledData.csv"
outputPath = "predictionsLRPipelineText/"

In [3]:
# *************************
# Training step
# *************************

# Create a DataFrame from trainingData.csv
# Training data in raw format
trainingData = spark.read.load(trainingData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [4]:
#trainingData.show()

In [5]:
# Configure an ML pipeline, which consists of five stages: 
# tokenizer -> split sentences in set of words
# remover -> remove stopwords
# hashingTF -> map set of words to a fixed-length feature vectors  (each 
# word becomes a feature and the value of the feature is the frequency of
#  the word in the sentence)
# idf -> compute the idf component of the TF-IDF measure
# lr -> logistic regression classification algorithm

In [6]:
# The Tokenizer splits each sentence in a set of words.
# It analyzes the content of column "text" and adds the 
# new column "words" in the returned DataFrame
tokenizer = Tokenizer().setInputCol("text").setOutputCol("words")

In [7]:
#tokenizedDF = tokenizer.transform(trainingData)
#tokenizedDF.printSchema()
#tokenizedDF.show(truncate=False)

In [8]:
# Remove stopwords.
# The StopWordsRemover component returns a new DataFrame with 
# a new column called "filteredWords". "filteredWords" is generated 
# by removing the stopwords from the content of column "words" 
#remover = StopWordsRemover().setInputCol("words").setOutputCol("filteredWords")
remover = StopWordsRemover()\
.setInputCol("words")\
.setOutputCol("filteredWords")

In [9]:
#stopwordRemovedDF = remover.transform(tokenizedDF)
#stopwordRemovedDF.printSchema()
#stopwordRemovedDF.show(truncate=False)

In [10]:
# Map words to a features
# Each word in filteredWords must become a feature in a Vector object
# The HashingTF Transformer can be used to perform this operation.
# This operations is based on a hash function and can potentially 
# map two different words to the same "feature". The number of conflicts
# in influenced by the value of the numFeatures parameter.  
# The "feature" version of the words is stored in Column "rawFeatures". 
# Each feature, for a document, contains the number of occurrences 
# of that feature in the document (TF component of the TF-IDF measure) 
hashingTF = HashingTF()\
.setNumFeatures(1000)\
.setInputCol("filteredWords")\
.setOutputCol("rawFeatures")

In [11]:
#TFDF = hashingTF.transform(stopwordRemovedDF)
#TFDF.printSchema()
#TFDF.show(truncate=False)

In [12]:
# Apply the IDF transformation/computation.
# Update the weight associated with each feature by considering also the 
# inverse document frequency component. The returned new column 
# is called "features", that is the standard name for the column that 
# contains the  predictive features used to create a classification model 
idf = IDF()\
.setInputCol("rawFeatures")\
.setOutputCol("features")

In [13]:
#TFIDFDF = idf.fit(TFDF).transform(TFDF)
#TFIDFDF.printSchema()
#TFIDFDF.show(truncate=False)

In [14]:
# Create a classification model based on the logistic regression algorithm
# We can set the values of the parameters of the 
# Logistic Regression algorithm using the setter methods.
lr = LogisticRegression()\
.setMaxIter(10)\
.setRegParam(0.01)

In [15]:
# Define the pipeline that is used to create the logistic regression
# model on the training data.
# In this case the pipeline is composed of five steps
# - text tokenizer
# - stopword removal
# - TF-IDF computation (performed in two steps)
# - Logistic regression model generation
pipeline = Pipeline().setStages([tokenizer, remover, hashingTF, idf, lr])

In [16]:
# Execute the pipeline on the training data to build the 
# classification model
classificationModel = pipeline.fit(trainingData)

# Now, the classification model can be used to predict the class label
# of new unlabeled data

In [17]:
# *************************
# Prediction  step
# *************************
# Read unlabeled data
# Create a DataFrame from unlabeledData.csv
# Unlabeled data in raw format
unlabeledData = spark.read.load(unlabeledData,\
                     format="csv", header=True, inferSchema=True)

In [18]:
#unlabeledData.printSchema()
#unlabeledData.show()

In [19]:
# Make predictions on unlabeled documents by using the 
# Transformer.transform() method.
# The transform will only use the 'features' columns
predictionsDF = classificationModel.transform(unlabeledData)

In [20]:
#predictionsDF.printSchema()
#predictionsDF.show()

In [21]:
# The returned DataFrame has the following schema (attributes)
# |-- label: string (nullable = true)
# |-- text: string (nullable = true)
# |-- words: array (nullable = true)
# |    |-- element: string (containsNull = true)
# |-- filteredWords: array (nullable = true)
# |    |-- element: string (containsNull = true)
# |-- rawFeatures: vector (nullable = true)
# |-- features: vector (nullable = true)
# |-- rawPrediction: vector (nullable = true)
# |-- probability: vector (nullable = true)
# |-- prediction: double (nullable = false)

# Select only the original features (i.e., the value of the original text attribute) and 
# the predicted class for each record
predictions = predictionsDF.select("text", "prediction")

In [22]:
#predictions.printSchema()
#predictions.show(truncate=False)

In [23]:
# Save the result in an HDFS output folder
predictions.write.csv(outputPath, header="true")